In [4]:
# -*- coding: utf-8 -*-
"""
@author: Gemini (三欄式日期輸入 修正版)
@date: 2025-10-12
@description: 
    此腳本為「交屋稅費分算計算機」的介面優化版本。
    v2.0 更新日誌:
    1.  [錯誤修正] 修正因寫入合併儲存格 (MergedCell) 導致的 'AttributeError'。
    2.  [介面優化] 將所有日期輸入從 YYYMMDD 單一儲存格，改為更直覺的「年、月、日」三欄式輸入。
    3.  [體驗升級] 為「月」與「日」欄位加入下拉式選單，提供防呆功能，避免使用者輸入錯誤格式。
    4.  [兼容性] 此版本為純 Excel 公式與樣式，不含任何 VBA，確保在所有平台與版本都能完美運作。
"""
import openpyxl
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.comments import Comment


def apply_styles_to_range(ws, cell_range, styles):
    """輔助函式：對指定的儲存格範圍一次性套用多種樣式。"""
    valid_styles = ['font', 'fill', 'alignment', 'border', 'number_format']
    rows = ws[cell_range]
    if not isinstance(rows, tuple):
        rows = ((rows,),)
    for row in rows:
        for cell in row:
            for style_name, style_obj in styles.items():
                if style_name in valid_styles and style_obj is not None:
                    setattr(cell, style_name, style_obj)


def create_final_workbook_three_column_date(filename="交屋稅費計算機_三欄式輸入版.xlsx"):
    """
    建立、格式化並儲存採用「三欄式日期輸入」的稅費計算機 Excel 檔案。
    """
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "專業稅費分算計算機"

    # --- 設定檔案的核心屬性 ---
    wb.properties.creator = "陳定康"
    wb.properties.title = "不動產買賣稅費分算計算機"
    wb.properties.description = "此工具用於精確計算不動產買賣過程中，買賣雙方應分攤之地價稅、房屋稅及其他相關費用。"

    # --- 樣式定義 ---
    fill_yellow_input = PatternFill(
        start_color='FFFFE0', end_color='FFFFE0', fill_type='solid')
    fill_blue_input = PatternFill(
        start_color='EAF1FF', end_color='EAF1FF', fill_type='solid')
    fill_title_green = PatternFill(
        start_color='D7E9D1', end_color='D7E9D1', fill_type='solid')
    fill_header_grey = PatternFill(
        start_color='E7E6E6', end_color='E7E6E6', fill_type='solid')
    fill_result_orange = PatternFill(
        start_color='FFF2E6', end_color='FFF2E6', fill_type='solid')
    fill_period_land = PatternFill(
        start_color='E2EFDA', end_color='E2EFDA', fill_type='solid')
    fill_period_house = PatternFill(
        start_color='DEEBF7', end_color='DEEBF7', fill_type='solid')
    font_title = Font(name='微軟正黑體', size=18, bold=True)
    font_section_header = Font(name='微軟正黑體', size=14, bold=True)
    font_body = Font(name='微軟正黑體', size=12)
    font_body_bold = Font(name='微軟正黑體', size=12, bold=True)
    font_buyer_pays = Font(name='微軟正黑體', size=12, bold=True, color="0000FF")
    font_seller_pays = Font(name='微軟正黑體', size=12, bold=True, color="C00000")
    font_red_bold = Font(name='微軟正黑體', size=12, bold=True, color="C00000")
    font_final_result = Font(name='微軟正黑體', size=16, bold=True, color="C00000")
    font_note = Font(name='微軟正黑體', size=11, italic=True)
    font_footer = Font(name='微軟正黑體', size=10, color="808080")
    align_center = Alignment(
        horizontal='center', vertical='center', wrap_text=True)
    align_left = Alignment(
        horizontal='left', vertical='center', wrap_text=True)
    align_right = Alignment(
        horizontal='right', vertical='center', wrap_text=True)
    align_left_top = Alignment(
        horizontal='left', vertical='top', wrap_text=True)
    thin_border = Border(left=Side(style='thin'), right=Side(
        style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    medium_border = Border(left=Side(style='medium'), right=Side(
        style='medium'), top=Side(style='medium'), bottom=Side(style='medium'))

    # --- 欄寬與列高設定 ---
    for col, width in {'A': 28, 'B': 10, 'C': 10, 'D': 10, 'E': 20, 'F': 55, 'G': 20}.items():
        ws.column_dimensions[col].width = width
    for i in range(1, 50):
        ws.row_dimensions[i].height = 32
    for row, height in {1: 50, 16: 45, 37: 55}.items():  # 行號+1
        ws.row_dimensions[row].height = height
    ws.row_dimensions[4].height = 130  # 行號+1
    ws.row_dimensions[17].height = 160  # 行號+1
    ws.row_dimensions[18].height = 160  # 行號+1

    # --- [核心修正] 因為插入新列，所有行號都必須 +1 ---
    # 為方便管理，我們定義一個行號偏移量
    ROW_OFFSET = 1

    # --- 輔助計算區公式更新 ---
    ws.column_dimensions['AA'].hidden = True
    ws.column_dimensions['AB'].hidden = True
    ws['AA1'] = f'=IF(AND(B{3}>0, C{3}>0, D{3}>0), DATE(B{3}+1911, C{3}, D{3}), "")'
    ws['AA2'] = f'=IF(AND(B{4}>0, C{4}>0, D{4}>0), DATE(B{4}+1911, C{4}, D{4}), "")'
    ws['AA3'] = f'=IFERROR(IF(C{8}<>"", (LEFT(C{8}, FIND("/", C{8})-1) / MID(C{8}, FIND("/", C{8})+1, LEN(C{8}))) * 100, B{8}), B{8})'
    ws['AA4'] = f'=IF(E{5}="是", 1, 0)'
    ws['AA5'] = f'=IF(AA2="","",AND(DAY(EOMONTH(AA2,0))=DAY(AA2), E{5}="否"))'
    ws['AA6'] = f'=IF(AND(B{24}>0, C{24}>0, D{24}>0), DATE(B{24}+1911, C{24}, D{24}), "")'
    ws['AA7'] = f'=IF(AND(B{25}>0, C{25}>0, D{25}>0), DATE(B{25}+1911, C{25}, D{25}), "")'
    ws['AA8'] = f'=IFERROR(IF(ISNUMBER(FIND("-",B{10})), VALUE(LEFT(B{10}, FIND("-",B{10})-1)), B{10}), B{10})'
    ws['AA9'] = f'=IFERROR(IF(ISNUMBER(FIND("-",B{10})), VALUE(MID(B{10}, FIND("-",B{10})+1, LEN(B{10}))), 0), 0)'
    ws['AA10'] = f'=IF(ISNUMBER(B{10}), B{10}, AA8-AA9)'

    # --- 核心公式定義 (所有行號都已 +1) ---
    calc_formulas = {
        'AB1': f'=IF(B{13}<>"", B{13}, IFERROR(ROUND(B{6}*B{7}*(AA3/100)*VLOOKUP(B{9},稅率表!$D$2:$E$3,2,FALSE),0),""))',
        'AB2': f'=IF(B{14}<>"", B{14}, IFERROR(ROUND(AA10*VLOOKUP(B{11}, 稅率表!$A$2:$B$14, 2, FALSE),0), ""))',
        'AB3': '=IF(AA5, 12, IFERROR(DATE(YEAR(AA2),12,31)-DATE(YEAR(AA2),1,1)+1, ""))',
        'AB4': '=IFERROR(DATE(YEAR(AA2)-IF(MONTH(AA2)<7,1,0),7,1), "")', 'AB5': '=IFERROR(DATE(YEAR(AA2)-IF(MONTH(AA2)<7,1,0)+1,6,30), "")',
        'AB6': '=IF(AA5, 12, IFERROR(AB5-AB4+1, ""))', 'AB7': '=IFERROR(EOMONTH(DATE(YEAR(AB5),2,1),0), "")',
        'AB8': '=IFERROR(IF(AA1>DATE(YEAR(AA2),8,31), "賣方", "買方"), "")', 'AB9': '=IFERROR(IF(AA1>AB7, "賣方", "買方"), "")',
        'AB10': '=IFERROR(IF(AA5, MONTH(AA2), AA2-DATE(YEAR(AA2),1,1)+1-AA4),"")', 'AB11': '=IFERROR(IF(AA5, 12-MONTH(AA2), DATE(YEAR(AA2),12,31)-AA2+AA4),"")',
        'AB12': '=IFERROR(IF(AA5, IF(MONTH(AA2)>=7, MONTH(AA2)-6, MONTH(AA2)+6), AA2-AB4+1-AA4),"")', 'AB13': '=IFERROR(IF(AA5, 12 - IF(MONTH(AA2)>=7, MONTH(AA2)-6, MONTH(AA2)+6), AB5-AA2+AA4),"")',
        'AB21': f'=IFERROR(IF(AND(B{21}>0, AA6>=AA2), IF((YEAR(AA6)=YEAR(AA2))*(MONTH(AA6)=MONTH(AA2)), ROUND(B{21}*(AA6-AA2+AA4)/DAY(EOMONTH(AA2,0)),0), ROUND((B{21}*(DAY(EOMONTH(AA2,0))-DAY(AA2)+AA4)/DAY(EOMONTH(AA2,0))) + (B{21}*MAX(0,(YEAR(AA6)-YEAR(AA2))*12+MONTH(AA6)-MONTH(AA2)-1)) + (B{21}*DAY(AA6)/DAY(EOMONTH(AA6,0))),0)), 0), 0)',
        'AB22': f'=IFERROR(IF(AND(B{22}>0, AA7>=AA2), IF((YEAR(AA7)=YEAR(AA2))*(MONTH(AA7)=MONTH(AA2)), ROUND(B{22}*(AA7-AA2+AA4)/DAY(EOMONTH(AA2,0)),0), ROUND((B{22}*(DAY(EOMONTH(AA2,0))-DAY(AA2)+AA4)/DAY(EOMONTH(AA2,0))) + (B{22}*MAX(0,(YEAR(AA7)-YEAR(AA2))*12+MONTH(AA7)-MONTH(AA2)-1)) + (B{22}*DAY(AA7)/DAY(EOMONTH(AA7,0))),0)), 0), 0)',
        'AB24': f'=IFERROR(IF(AND(B{21}>0, AA6<AA2), ROUND((B{21}*(DAY(EOMONTH(AA6,0))-DAY(AA6))/DAY(EOMONTH(AA6,0))) + (B{21}*MAX(0,(YEAR(AA2)-YEAR(AA6))*12+MONTH(AA2)-MONTH(AA6)-1)) + (B{21}*DAY(AA2)/DAY(EOMONTH(AA2,0))),0), 0), 0)',
        'AB25': f'=IFERROR(IF(AND(B{22}>0, AA7<AA2), ROUND((B{22}*(DAY(EOMONTH(AA7,0))-DAY(AA7))/DAY(EOMONTH(AA7,0))) + (B{22}*MAX(0,(YEAR(AA2)-YEAR(AA7))*12+MONTH(AA2)-MONTH(AA7)-1)) + (B{22}*DAY(AA2)/DAY(EOMONTH(AA2,0))),0), 0), 0)',
        f'B{17}': '=IF(AB10<>"", AB10 & IF(AA5, " 月", " 天"), "")', f'C{17}': '=IF(AB11<>"", AB11 & IF(AA5, " 月", " 天"), "")', f'D{17}': '=AB1',
        f'E{17}': '=IFERROR(IF(AB8="賣方", ROUND(D17*AB11/AB3,0), ROUND(D17*AB10/AB3,0)),0)',
        f'F{17}': f'=IF(E{17}=0, "無需找補", IF(B{13}<>"", "納稅人為【" & AB8 & "】，" & IF(AB8="賣方", "由【買方】補貼賣方", "由【賣方】補貼買方") & "。" & CHAR(10) & "【找補計算】" & CHAR(10) & "   " & TEXT(D{17}, "#,##0") & " (年稅額) * (" & IF(AB8="賣方", AB11, AB10) & " / " & AB3 & ") (持有比例) = " & TEXT(E{17}, "#,##0") & CHAR(10), "納稅人為【" & AB8 & "】，" & IF(AB8="賣方", "由【買方】補貼賣方", "由【賣方】補貼買方") & "。" & CHAR(10) & "【年稅額計算】" & CHAR(10) & "   地價總額 = " & TEXT(B{6},"#,##0") & " * " & TEXT(B{7},"#,##0") & " * " & TEXT(AA3/100,"0.00%") & " = " & TEXT(ROUND(B{6}*B{7}*(AA3/100),0),"#,##0") & CHAR(10) & "   應繳地價稅 = " & TEXT(ROUND(B{6}*B{7}*(AA3/100),0),"#,##0") & " * " & TEXT(VLOOKUP(B{9},稅率表!$D$2:$E$3,2,FALSE),"0.0%") & " = " & TEXT(D{17},"#,##0") & CHAR(10) & "【找補計算】" & CHAR(10) & "   " & TEXT(D{17}, "#,##0") & " (年稅額) * (" & IF(AB8="賣方", AB11, AB10) & " / " & AB3 & ") (持有比例) = " & TEXT(E{17}, "#,##0") & CHAR(10)) & "【款項性質】" & CHAR(10) & "   " & "此為民國 " & YEAR(AA2)-1911 & " 年度地價稅找補，屬【" & IF(AA2>DATE(YEAR(AA2),8,31), "未到期數", "已到期數") & "】款項。" & CHAR(10) & "---" & CHAR(10) & "【稅務規則】" & CHAR(10) & "● 課稅年度： 民國 " & YEAR(AA2)-1911 & " 年 (1/1 ~ 12/31)" & CHAR(10) & "● 開徵期間： 民國 " & YEAR(AA2)-1911 & " 年 11 月 1 日 至 11 月 30 日" & CHAR(10) & "● 納稅基準日： 民國 " & YEAR(AA2)-1911 & " 年 8 月 31 日" & CHAR(10) & "● 本案納稅人： 【" & AB8 & "】 (以基準日之地政登記所有權人為準)")',
        f'B{18}': '=IF(AB12<>"", AB12 & IF(AA5, " 月", " 天"), "")', f'C{18}': '=IF(AB13<>"", AB13 & IF(AA5, " 月", " 天"), "")', f'D{18}': '=AB2',
        f'E{18}': '=IFERROR(IF(AB9="賣方", ROUND(D18*AB13/AB6,0), ROUND(D18*AB12/AB6,0)),0)',
        f'F{18}': f'=IF(E{18}=0, "無需找補", IF(B{14}<>"", "納稅人為【" & AB9 & "】，" & IF(AB9="賣方", "由【買方】補貼賣方", "由【賣方】補貼買方") & "。" & CHAR(10) & "【找補計算】" & CHAR(10) & "   " & TEXT(D{18}, "#,##0") & " (年稅額) * (" & IF(AB9="賣方", AB13, AB12) & " / " & AB6 & ") (持有比例) = " & TEXT(E{18}, "#,##0") & CHAR(10), "納稅人為【" & AB9 & "】，" & IF(AB9="賣方", "由【買方】補貼賣方", "由【賣方】補貼買方") & "。" & CHAR(10) & "【年稅額計算】" & CHAR(10) & "   應繳房屋稅 = " & IF(AA9>0, "(" & TEXT(AA8,"#,##0") & "-" & TEXT(AA9,"#,##0") & ")", TEXT(AA10,"#,##0")) & " * " & TEXT(D{11},"0.0%") & " = " & TEXT(D{18}, "#,##0") & CHAR(10) & "【找補計算】" & CHAR(10) & "   " & TEXT(D{18}, "#,##0") & " (年稅額) * (" & IF(AB9="賣方", AB13, AB12) & " / " & AB6 & ") (持有比例) = " & TEXT(E{18}, "#,##0") & CHAR(10)) & "【款項性質】" & CHAR(10) & "   " & "此為民國 " & YEAR(AB5)-1911 & " 年度房屋稅找補，屬【" & IF(AA2>AB7, "未到期數", "已到期數") & "】款項。" & CHAR(10) & "---" & CHAR(10) & "【稅務規則】" & CHAR(10) & "● 課稅期間： 民國 " & YEAR(AB4)-1911 & "/7/1 至 " & YEAR(AB5)-1911 & "/6/30" & CHAR(10) & "● 開徵期間： 民國 " & YEAR(AB5)-1911 & " 年 5 月 1 日 至 5 月 31 日" & CHAR(10) & "● 納稅基準日： " & TEXT(AB7, "民國 e 年 m 月 d 日") & CHAR(10) & "● 本案納稅人： 【" & AB9 & "】 (以基準日之房屋所有權人為準)")',
        f'D{26}': f'=ABS(SUM(C{33}:C{35})-SUM(D{33}:D{35}))',
        f'A{27}': f'=IF(SUM(C{33}:C{35})>SUM(D{33}:D{35}), "【買方】應補貼【賣方】 NT$ " & TEXT(D{26}, "#,##0") & " 元", IF(SUM(D{33}:D{35})>SUM(C{33}:C{35}), "【賣方】應補貼【買方】 NT$ " & TEXT(D{26}, "#,##0") & " 元", "管理費、車位費、押金無需找補"))',
        f'C{31}': f'=IF(AB8="賣方", E{17}, 0)', f'D{31}': f'=IF(AB8="買方", E{17}, 0)', f'C{32}': f'=IF(AB9="賣方", E{18}, 0)', f'D{32}': f'=IF(AB9="買方", E{18}, 0)',
        f'C{33}': '=AB21', f'D{33}': '=AB24', f'C{34}': '=AB22', f'D{34}': '=AB25',
        f'E{33}': f'=IF(C{33}>0, "月費 " & TEXT(B{21}, "#,##0") & " 元，賣方已預繳至 " & TEXT(AA6, "e/m/d") & "，依交屋後買方應持有之 " & TEXT(AA6-AA2+AA4, "0") & " 天計算。", IF(D{33}>0, "月費 " & TEXT(B{21}, "#,##0") & " 元，賣方僅繳納至 " & TEXT(AA6, "e/m/d") & "，依交屋前賣方應持有之 " & TEXT(AA2-AA6, "0") & " 天計算補繳。", "無需找補"))',
        f'E{34}': f'=IF(C{34}>0, "月費 " & TEXT(B{22}, "#,##0") & " 元，賣方已預繳至 " & TEXT(AA7, "e/m/d") & "，依交屋後買方應持有之 " & TEXT(AA7-AA2+AA4, "0") & " 天計算。", IF(D{34}>0, "月費 " & TEXT(B{22}, "#,##0") & " 元，賣方僅繳納至 " & TEXT(AA7, "e/m/d") & "，依交屋前賣方應持有之 " & TEXT(AA2-AA7, "0") & " 天計算補繳。", "無需找補"))',
        f'C{35}': 0, f'D{35}': f'=IF(B{23}<>"", B{23}, 0)',
        f'C{36}': f'=SUM(C{31}:C{35})', f'D{36}': f'=SUM(D{31}:D{35})',
        f'B{37}': f'=IF(C{36}>D{36}, "【買方】應支付給【賣方】 NT$ " & TEXT(C{36}-D{36}, "#,##0"), IF(D{36}>C{36}, "【賣方】應支付給【買方】 NT$ " & TEXT(D{36}-C{36}, "#,##0"), "雙方無需找補"))'
    }
    for cell_ref, formula in calc_formulas.items():
        ws[cell_ref] = formula

    # --- 介面佈局 ---
    ws.merge_cells('A1:G1')
    ws['A1'] = "不動產買賣稅費分算計算機"
    apply_styles_to_range(ws, 'A1', {
                          'font': font_title, 'fill': fill_title_green, 'alignment': align_center})

    # --- [核心修改] 新增日期欄位標題列 ---
    ws.cell(row=2, column=2, value="民國年")
    ws.cell(row=2, column=3, value="月")
    ws.cell(row=2, column=4, value="日")
    apply_styles_to_range(
        ws, 'B2:D2', {'font': font_body_bold, 'alignment': align_center})

    # --- 日期輸入區塊 (行號+1) ---
    ws[f'A{3}'] = "權狀登記(過戶)日期"
    ws[f'A{4}'] = "房屋點交(交屋)日期"
    ws[f'A{5}'] = "是否帶租約交屋"
    apply_styles_to_range(ws, f'A3:A5', {
                          'font': font_body, 'fill': fill_header_grey, 'alignment': align_right})

    ws[f'B{3}'], ws[f'C{3}'], ws[f'D{3}'] = 114, 10, 1
    ws[f'B{4}'], ws[f'C{4}'], ws[f'D{4}'] = 114, 10, 31
    ws[f'E{5}'] = "否"

    apply_styles_to_range(ws, f'B3:D4', {
                          'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    apply_styles_to_range(ws, f'E5', {
                          'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})

    dv_lease = DataValidation(type="list", formula1='"是,否"')
    dv_lease.add(f'E{5}')
    ws.add_data_validation(dv_lease)

    ws.merge_cells(f'E{3}:G{3}')
    ws.merge_cells(f'E{4}:G{4}')
    ws[f'E{3}'] = '=IF(AA1<>"", TEXT(AA1, "民國 e 年 m 月 d 日"), "請輸入有效的年/月/日")'
    ws[f'E{4}'] = '=IF(AA2<>"", TEXT(AA2, "民國 e 年 m 月 d 日"), "請輸入有效的年/月/日")'
    apply_styles_to_range(
        ws, f'E3:E4', {'font': font_body, 'alignment': align_left})

    ws.merge_cells(f'F{5}:G{5}')
    ws[f'F{5}'] = f'=IF(AA5, "【按月分算模式】", "【按日分算模式】") & " 交屋日歸屬：" & IF(E{5}="是", "買方", "賣方")'
    apply_styles_to_range(
        ws, f'F{5}', {'font': font_body_bold, 'alignment': align_left})

    # --- 其他資訊區塊 (所有行號都已 +1) ---
    ws.merge_cells(f'A{6}:A{11}')
    ws[f'A{6}'] = "模式一：詳細資料輸入"
    apply_styles_to_range(ws, f'A{6}', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    for cell_ref, label in {f"E{6}": "申報地價 (元/m²)", f"E{7}": "土地面積 (m²)", f"E{8}": "權利範圍 (% 或 分數)", f"E{9}": "地價稅率", f"E{10}": "房屋現值 (元)", f"E{11}": "房屋使用情境"}.items():
        ws.merge_cells(f'{cell_ref[0]}{cell_ref[1:]}:G{cell_ref[1:]}')
        ws[cell_ref] = label
        apply_styles_to_range(
            ws, cell_ref, {'font': font_body, 'alignment': align_left})
    for row_num in [6, 7, 9, 10]:
        ws.merge_cells(f'B{row_num}:D{row_num}')
    ws.merge_cells(f'C{8}:D{8}')
    ws.merge_cells(f'B{11}:C{11}')
    apply_styles_to_range(ws, f'B6:D11', {
                          'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    ws[f'B{6}'] = 155091.2
    ws[f'B{7}'] = 449
    ws[f'B{8}'] = 1.28
    ws[f'B{8}'].number_format = '0.00"%"'
    ws[f'C{8}'] = ""
    ws[f'B{9}'] = "一般用地 (千分之十)"
    ws[f'B{10}'] = '13272200-277300'
    ws[f'B{11}'] = "自住用-全國3戶內"
    ws[f'D{11}'] = f'=IFERROR(VLOOKUP(B{11}, 稅率表!$A$2:$B$14, 2, FALSE), "")'
    ws[f'D{11}'].number_format = '0.00%'

    ws.merge_cells(f'A{13}:A{14}')
    ws[f'A{13}'] = "模式二：快速稅額輸入"
    apply_styles_to_range(ws, f'A{13}', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    ws.merge_cells(f'B{13}:D{13}')
    ws.merge_cells(f'B{14}:D{14}')
    apply_styles_to_range(ws, f'B13:B14', {
                          'font': font_body, 'fill': fill_blue_input, 'alignment': align_center})
    ws[f'E{13}'] = "年度應納地價稅 (快速)"
    ws[f'E{14}'] = "年度應納房屋稅 (快速)"
    apply_styles_to_range(
        ws, f'E13:E14', {'font': font_body, 'alignment': align_left})
    ws.merge_cells(f'F{13}:G{14}')
    ws[f'F{13}'] = "若填寫此區藍色欄位，將優先採用此處稅額進行計算。"
    apply_styles_to_range(
        ws, f'F{13}', {'font': font_note, 'alignment': align_center})

    ws.merge_cells(f'A{15}:G{15}')
    ws[f'A{15}'] = "（二）核心稅費計算"
    apply_styles_to_range(ws, f'A{15}', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    for col_idx, header in enumerate(["項目", "賣方持有期間", "買方持有期間", "年度總稅額", "找補金額", "備註 (計算過程 & 稅務規則)"], 1):
        ws.cell(row=16, column=col_idx, value=header)
    ws.merge_cells(f'F{16}:G{16}')
    apply_styles_to_range(ws, f'A16:G16', {
                          'font': font_body_bold, 'fill': fill_header_grey, 'alignment': align_center})
    ws[f'A{17}'] = "地價稅"
    ws[f'A{18}'] = "房屋稅"
    apply_styles_to_range(
        ws, f'A17:A18', {'font': font_body, 'alignment': align_center})
    apply_styles_to_range(ws, f'B17:C17', {
                          'font': font_body, 'alignment': align_center, 'fill': fill_period_land})
    apply_styles_to_range(ws, f'B18:C18', {
                          'font': font_body, 'alignment': align_center, 'fill': fill_period_house})
    ws.merge_cells(f'F{17}:G{17}')
    ws.merge_cells(f'F{18}:G{18}')
    apply_styles_to_range(
        ws, f'F17:F18', {'font': font_body, 'alignment': align_left_top})
    apply_styles_to_range(ws, f'D17:E18', {
                          'font': font_body, 'alignment': align_center, 'number_format': '#,##0'})

    ws.merge_cells(f'A{20}:G{20}')
    ws[f'A{20}'] = "（三）其他費用分算"
    apply_styles_to_range(ws, f'A{20}', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    ws[f'A{21}'] = "管理費(月)"
    ws[f'A{22}'] = "車位費(月)"
    ws[f'A{23}'] = "押金/保證金"
    apply_styles_to_range(ws, f'A21:A23', {
                          'font': font_body, 'fill': fill_header_grey, 'alignment': align_right})
    ws.merge_cells(f'B{21}:D{21}')
    ws.merge_cells(f'B{22}:D{22}')
    ws.merge_cells(f'B{23}:D{23}')
    apply_styles_to_range(ws, f'B21:D23', {
                          'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    ws[f'B{21}'] = 4430
    ws[f'B{22}'] = 1200
    ws[f'B{23}'] = 0
    ws[f'A{24}'] = "管理費已預繳至"
    ws[f'A{25}'] = "車位費已預繳至"
    apply_styles_to_range(ws, f'A24:A25', {
                          'font': font_body, 'fill': fill_header_grey, 'alignment': align_right})
    apply_styles_to_range(ws, f'B24:D25', {
                          'font': font_body, 'fill': fill_yellow_input, 'alignment': align_center})
    ws[f'B{24}'], ws[f'C{24}'], ws[f'D{24}'] = 114, 10, 31
    ws[f'B{25}'], ws[f'C{25}'], ws[f'D{25}'] = 114, 10, 31
    ws[f'E{24}'] = '=IF(AA6<>"", TEXT(AA6, "e/m/d"), "")'
    ws[f'E{25}'] = '=IF(AA7<>"", TEXT(AA7, "e/m/d"), "")'
    ws.merge_cells(f'E{21}:G{22}')
    ws[f'E{21}'] = "請填寫月繳金額，若無則填 0。\n找補金額將依交屋日與費用預繳截止日，按當月天數比例分算。"
    apply_styles_to_range(
        ws, f'E{21}', {'font': font_note, 'alignment': align_left_top})
    ws.merge_cells(f'E{23}:G{23}')
    ws[f'E{23}'] = "此金額將作為「賣方應付給買方」的款項"
    apply_styles_to_range(
        ws, f'E{23}', {'font': font_note, 'alignment': align_left})

    ws.merge_cells(f'A{26}:C{26}')
    ws[f'A{26}'] = "其他費用找補金額 (管理費、清潔費、押金)"
    apply_styles_to_range(
        ws, f'A{26}', {'font': font_body_bold, 'alignment': align_center})
    ws.merge_cells(f'D{26}:G{26}')
    apply_styles_to_range(ws, f'D{26}', {
                          'font': font_body_bold, 'number_format': '#,##0', 'alignment': align_center})
    ws.merge_cells(f'A{27}:G{27}')
    apply_styles_to_range(
        ws, f'A{27}', {'font': font_body, 'alignment': align_center})

    ws.merge_cells(f'A{29}:G{29}')
    ws[f'A{29}'] = "（四）最終結算"
    apply_styles_to_range(ws, f'A{29}', {
                          'font': font_section_header, 'fill': fill_header_grey, 'alignment': align_center})
    ws.merge_cells(f'A{30}:B{30}')
    ws[f'A{30}'] = "項目"
    ws[f'C{30}'] = "買方應付給賣方"
    ws[f'D{30}'] = "賣方應付給買方"
    ws.merge_cells(f'E{30}:G{30}')
    ws[f'E{30}'] = "說明"
    apply_styles_to_range(ws, f'A30:G30', {
                          'font': font_body_bold, 'fill': fill_header_grey, 'alignment': align_center})
    items = {31: ("地價稅找補", "依持有天數比例分算"), 32: ("房屋稅找補", "依持有天數比例分算"), 33: (
        "管理費找補", ""), 34: ("車位費找補", ""), 35: ("押金/保證金轉交", "賣方將原持有之押金轉交給新屋主(買方)")}
    for row, (label, desc) in items.items():
        ws.merge_cells(f'A{row}:B{row}')
        ws[f'A{row}'] = label
        ws.merge_cells(f'E{row}:G{row}')
        ws[f'E{row}'] = desc
    ws.merge_cells(f'A{36}:B{36}')
    ws[f'A{36}'] = "款項小計"
    ws.merge_cells(f'E{36}:G{36}')
    apply_styles_to_range(
        ws, f'A31:A36', {'font': font_body, 'alignment': align_left})
    apply_styles_to_range(
        ws, f'E31:E35', {'font': font_note, 'alignment': align_left_top})
    apply_styles_to_range(
        ws, f'A{36}', {'font': font_body_bold, 'alignment': align_center})
    apply_styles_to_range(
        ws, f'C31:D36', {'number_format': '#,##0', 'alignment': align_center})
    apply_styles_to_range(ws, f'C31:C36', {'font': font_buyer_pays})
    apply_styles_to_range(ws, f'D31:D36', {'font': font_seller_pays})
    ws.merge_cells(f'A{37}:A{38}')
    ws[f'A{37}'] = "最終結算結果"
    apply_styles_to_range(ws, f'A{37}', {
                          'font': font_section_header, 'fill': fill_result_orange, 'alignment': align_center})
    ws.merge_cells(f'B{37}:G{38}')
    apply_styles_to_range(ws, f'B{37}', {
                          'font': font_final_result, 'fill': fill_result_orange, 'alignment': align_center})
    ws.merge_cells(f'A{40}:G{40}')
    ws[f'A{40}'] = "本計算機由 陳定康 設計"
    apply_styles_to_range(
        ws, f'A{40}', {'font': font_footer, 'alignment': align_right})

    comments = {f'B{3}': '請輸入民國年。範例：114', f'B{10}': '請輸入房屋評定現值總額。\n【新功能】可使用減法格式輸入：\n[總現值]-[免稅現值]', f'C{8}': '請輸入分數，例如：1/4。\n若此欄位與左側百分比欄位皆有數值，將【優先採用此分數】進行計算。',
                f'B{13}': '若您已知曉年度總稅額，可直接填寫於此。\n系統將【優先採用此處稅額】，並忽略上方「模式一」的詳細資料。', f'B{14}': '若您已知曉年度總稅額，可直接填寫於此。\n系統將【優先採用此處稅額】，並忽略上方「模式一」的詳細資料。'}
    for cell_ref, comment_text in comments.items():
        ws[cell_ref].comment = Comment(comment_text, "陳定康")

    for r in [f'A{3}:G{5}', f'A{6}:G{11}', f'A{13}:G{14}', f'A{15}:G{18}', f'A{20}:G{27}', f'A{29}:G{38}']:
        apply_styles_to_range(ws, r, {'border': thin_border})
    apply_styles_to_range(ws, f'A{37}:G{38}', {'border': medium_border})

    dv_land = DataValidation(type="list", formula1="=稅率表!$D$2:$D$3")
    dv_land.add(f'B{9}')
    ws.add_data_validation(dv_land)
    dv_house = DataValidation(type="list", formula1="=稅率表!$A$2:$A$14")
    dv_house.add(f'B{11}')
    ws.add_data_validation(dv_house)

    dv_date = DataValidation(type="custom", formula1="=AA2>=AA1")
    dv_date.errorTitle = "日期輸入錯誤"
    dv_date.error = f"【房屋點交日期】(A{4}) 必須晚於或等於【權狀登記日期】(A{3})。\n\n請重新輸入正確的日期。"
    dv_date.add(f'D{4}')
    ws.add_data_validation(dv_date)

    dv_month = DataValidation(
        type="list", formula1='"1,2,3,4,5,6,7,8,9,10,11,12"')
    ws.add_data_validation(dv_month)
    dv_day = DataValidation(
        type="list", formula1=f'"{",".join(map(str, range(1, 32)))}"')
    ws.add_data_validation(dv_day)
    for row in [3, 4, 24, 25]:
        dv_month.add(f'C{row}')
        dv_day.add(f'D{row}')

    ws2 = wb.create_sheet("稅率表")
    house_tax_data = [("房屋使用情境選項", "對應稅率"), ("自住用-全國單一自住(現值一定金額下)", 0.01), ("自住用-全國3戶內", 0.012), ("公益出租人/社會住宅", 0.012), ("非自住-出租(達租金標準)/繼承共有-4戶內", 0.015), ("非自住-出租(達租金標準)/繼承共有-5~6戶", 0.02),
                      ("非自住-出租(達租金標準)/繼承共有-7戶以上", 0.024), ("非自住-其他住家用-2戶內", 0.032), ("非自住-其他住家用-3~4戶", 0.038), ("非自住-其他住家用-5~6戶", 0.042), ("非自住-其他住家用-7戶以上", 0.048), ("營業用", 0.03), ("私人醫院/診所/事務所用", 0.03), ("非住家非營業用(人民團體等)", 0.02)]
    land_tax_data = [("地價稅率選項", "對應稅率"), ("自用住宅 (千分之二)",
                                          0.002), ("一般用地 (千分之十)", 0.010)]
    for r_idx, row_data in enumerate(house_tax_data, 1):
        for c_idx, cell_data in enumerate(row_data, 1):
            ws2.cell(row=r_idx, column=c_idx, value=cell_data)
    for r_idx, row_data in enumerate(land_tax_data, 1):
        for c_idx, cell_data in enumerate(row_data, 1):
            ws2.cell(row=r_idx, column=c_idx + 3, value=cell_data)
    apply_styles_to_range(ws2, 'A1:B1', {'font': font_body_bold})
    apply_styles_to_range(ws2, 'D1:E1', {'font': font_body_bold})
    apply_styles_to_range(ws2, 'B2:B14', {'number_format': '0.00%'})
    apply_styles_to_range(ws2, 'E2:E3', {'number_format': '0.000%'})
    ws2.column_dimensions['A'].width = 45
    ws2.column_dimensions['B'].width = 15
    ws2.column_dimensions['D'].width = 30
    ws2.column_dimensions['E'].width = 15

    try:
        wb.calculation.fullCalcOnLoad = True
        wb.save(filename)
        print(f"✅ 成功創建 Excel 檔案： '{filename}'")
        print("-" * 50)
        print("💡 三欄式日期輸入版 功能提示：")
        print("   - [介面升級] 所有日期輸入已改為更直覺的「年、月、日」三欄式。")
        print("   - [輸入防呆] 「月」與「日」欄位已加入下拉選單，避免格式錯誤。")
        print("   - [完全兼容] 檔案不含VBA，可在任何 Excel 環境下穩定使用。")
        print("-" * 50)
    except Exception as e:
        print(f"❌ 創建檔案時發生錯誤：{e}")


if __name__ == '__main__':
    create_final_workbook_three_column_date()

✅ 成功創建 Excel 檔案： '交屋稅費計算機_三欄式輸入版.xlsx'
--------------------------------------------------
💡 三欄式日期輸入版 功能提示：
   - [介面升級] 所有日期輸入已改為更直覺的「年、月、日」三欄式。
   - [輸入防呆] 「月」與「日」欄位已加入下拉選單，避免格式錯誤。
   - [完全兼容] 檔案不含VBA，可在任何 Excel 環境下穩定使用。
--------------------------------------------------


In [ ]:
' 強制宣告所有變數
Option Explicit

'
' ClearInputs_ProtectedSheet 巨集
'
' 功能：  在「受保護」的工作表上，安全地清除所有使用者輸入欄位。
' 版本：  v3.0 (防鎖定版)
' 核心：  1. [新增] 在執行操作前，使用密碼自動解除工作表保護。
'         2. [新增] 操作完成後或發生錯誤時，必定重新套用保護，確保公式安全。
'         3. [更新] 清除的儲存格清單已完全對應「三欄式日期輸入」的介面。
'
Public Sub ClearInputs_ProtectedSheet()

    ' --- 宣告變數 ---
    Dim ws As Worksheet
    Dim targetSheetName As String
    Dim cellsToClear As Variant
    Dim cellAddress As Variant
    Dim protectionPassword As String
    
    ' --- 【請在此處設定您的工作表保護密碼】 ---
    ' 如果您的工作表沒有密碼，請維持空字串 ""
    protectionPassword = "您的密碼" ' <== 請將 "您的密碼" 替換成您設定的真實密碼
    
    ' 將工作表名稱定義為變數
    targetSheetName = "專業稅費分算計算機"
    
    ' --- 【對應三欄式日期輸入的最終清單】 ---
    cellsToClear = Array( _
        "B3", "C3", "D3", "B4", "C4", "D4", "E5", _
        "B6", "B7", "B8", "C8", "B10", "B11", _
        "B13", "B14", _
        "B21", "B22", "B23", _
        "B24", "C24", "D24", "B25", "C25", "D25" _
    )
    
    ' --- 錯誤處理機制 ---
    On Error GoTo ErrorHandler

    ' --- 初始化物件 ---
    Set ws = ThisWorkbook.Sheets(targetSheetName)
    
    ' --- 主程式邏輯 ---
    Application.ScreenUpdating = False
    
    ' [核心修改] 執行前，解除工作表保護
    ws.Unprotect Password:=protectionPassword
    
    ' 遍歷陣列中的每一個儲存格地址
    For Each cellAddress In cellsToClear
        ' 使用 .Value = "" 來清除內容
        ws.Range(cellAddress).Value = ""
    Next cellAddress
    
    ' --- 收尾工作 ---
    ' 將游標移回第一個輸入欄位
    ws.Range("B3").Select
    
    ' [核心修改] 無論如何，都要在結束前重新保護工作表
    ws.Protect Password:=protectionPassword
    Application.ScreenUpdating = True
    
    MsgBox "指定欄位已成功清除！", vbInformation, "操作完成"
    
    Exit Sub

' --- 錯誤處理區塊 ---
ErrorHandler:
    ' [核心修改] 即使發生錯誤，也要確保工作表被重新鎖定
    If Not ws Is Nothing Then
        ws.Protect Password:=protectionPassword
    End If
    
    Application.ScreenUpdating = True
    
    If Err.Number = 9 Then
        MsgBox "執行失敗！" & vbCrLf & vbCrLf & "錯誤原因：找不到名為「" & targetSheetName & "」的工作表。", vbCritical, "錯誤"
    Else
        MsgBox "執行失敗，發生未預期的錯誤！" & vbCrLf & vbCrLf & "錯誤代碼：" & Err.Number & vbCrLf & "錯誤描述：" & Err.Description, vbCritical, "錯誤"
    End If
    
    Err.Clear

End Sub